# LETR Basic Usage Demo

In [ ]:
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200
import torchvision.transforms.functional as functional
import torch.nn.functional as F
from glob import glob
from models import build_model
from util.misc import nested_tensor_from_tensor_list
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from pathlib import Path

In [ ]:
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image):
        for t in self.transforms:
            image = t(image)
        return image

    def __repr__(self):
        format_string = self.__class__.__name__ + "("
        for t in self.transforms:
            format_string += "\n"
            format_string += "    {0}".format(t)
        format_string += "\n)"
        return format_string

class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, image):
        image = functional.normalize(image, mean=self.mean, std=self.std)
        return image

class ToTensor(object):
    def __call__(self, img):
        return functional.to_tensor(img)

def resize(image, size, max_size=None):
    # size can be min_size (scalar) or (w, h) tuple
    def get_size_with_aspect_ratio(image_size, size, max_size=None):
        w, h = image_size
        if max_size is not None:
            min_original_size = float(min((w, h)))
            max_original_size = float(max((w, h)))
            if max_original_size / min_original_size * size > max_size:
                size = int(round(max_size * min_original_size / max_original_size))
        if (w <= h and w == size) or (h <= w and h == size):
            return (h, w)
        if w < h:
            ow = size
            oh = int(size * h / w)
        else:
            oh = size
            ow = int(size * w / h)
        return (oh, ow)

    def get_size(image_size, size, max_size=None):
        if isinstance(size, (list, tuple)):
            return size[::-1]
        else:
            return get_size_with_aspect_ratio(image_size, size, max_size)

    size = get_size(image.size, size, max_size)
    rescaled_image = functional.resize(image, size)

    return rescaled_image

class Resize(object):
    def __init__(self, sizes, max_size=None):
        assert isinstance(sizes, (list, tuple))
        self.sizes = sizes
        self.max_size = max_size

    def __call__(self, img):
        size = self.sizes
        return resize(img, size, self.max_size)



In [ ]:

def scale_positions(lines, heatmap_scale=(128, 128), im_shape=None): # TODO: check if still works for non square heatmap
    if len(lines) == 0:
        return []
    fy, fx = heatmap_scale[1] / im_shape[0], heatmap_scale[0] / im_shape[1]

    lines[:, :, 0] = np.clip(lines[:, :, 0] * fx, 0, heatmap_scale[0] - 1e-4)
    lines[:, :, 1] = np.clip(lines[:, :, 1] * fy, 0, heatmap_scale[1] - 1e-4)
    return lines

## Load Model Pre-trained Weights

In [ ]:
ckpt_path = '/home/kallelis/PrimitiveExtraction/PrimitiveExtraction/Detection/LETR/exp/exp/res101_stage2_focal/checkpoints/'
npz_dir = ckpt_path + f'/eida_final_npz/'
# os.makedirs(npz_dir, exist_ok=True)

In [ ]:
# obtain checkpoints
checkpoint = torch.load(f'{ckpt_path}checkpoint0024.pth', map_location='cpu')

# load model
args = checkpoint['args']
model, _, postprocessors = build_model(args)
model.load_state_dict(checkpoint['model'])
model = model.eval()



In [ ]:
model = model.to('cuda')

In [ ]:
# load image


In [ ]:
full_path = Path("/home/kallelis/PrimitiveExtraction/PrimitiveExtraction/data/eida_final/images")
test_size = 1100
normalize = Compose([
        ToTensor(),
        Normalize([0.538, 0.494, 0.453], [0.257, 0.263, 0.273]),
        Resize([test_size]),
    ])

In [ ]:
for image_name in os.listdir(full_path): 
    print(image_name)
    break

In [ ]:
from PIL import Image


In [ ]:
import os
not_computed = os.listdir('/home/kallelis/PrimitiveExtraction/PrimitiveExtraction/data/eida_final/images')
computed=os.listdir('/home/kallelis/PrimitiveExtraction/PrimitiveExtraction/Detection/LETR/exp/exp/res101_stage2_focal/checkpoints/eida_final_npz')
remaining = list(set(not_computed) - set(computed))

In [ ]:
len(computed)

In [ ]:
for image_name in os.listdir(full_path): 
    # im_name ='ms2_0243_716,1556,778,407'
    im_name = image_name[:-4]
    if (f"{im_name}.npz" in computed):
        # print(f"skipping {im_name}")
        continue
    print(f"computing {im_name}")
    im_path = full_path / f'{im_name}.jpg'

    raw_img = Image.open(im_path).convert("RGB")
    w,h = raw_img.size
    # original = original[:,:,:3]
    # raw_img = original
    # h, w = raw_img.shape[0], raw_img.shape[1]
    orig_size = torch.as_tensor([int(h), int(w)])


    img = normalize(raw_img)
    inputs = nested_tensor_from_tensor_list([img])
    # inputs = inputs.to('cuda')
    # plt.axis('off')
    # plt.imshow(raw_img)
    with torch.no_grad():
        outputs = model(inputs)
    outputs = outputs[0]
    out_logits, out_line = outputs['pred_logits'], outputs['pred_lines']
    prob = F.softmax(out_logits, -1)
    scores, labels = prob[..., :-1].max(-1)
    img_h, img_w = orig_size.unbind(0)
    # scale_fct = torch.unsqueeze(torch.stack([img_w, img_h, img_w, img_h], dim=0), dim=0).to('cuda')
    scale_fct = torch.unsqueeze(torch.stack([img_w, img_h, img_w, img_h], dim=0), dim=0)

    lines = out_line * scale_fct[:, None, :]
    # lines = lines.view(1000, 2, 2)
    # lines = lines.flip([-1])# this is yxyx format
    scores = scores.detach().cpu().numpy()
    results = [
        {"scores": s, "labels": l, "lines": b}
        for s, l, b in zip(scores, labels, lines)
    ]

    pred_logits = outputs["pred_logits"]
    bz = pred_logits.shape[0]
    assert bz == 1
    query = pred_logits.shape[1]

    # rst = results[0]["lines"].detach()
    rst = results[0]["lines"]

    pred_lines = rst.view(query, 2, 2)

    # pred_lines = pred_lines.flip([-1])  # this is yxyx format

    h, w = orig_size.tolist()
    # pred_lines = scale_positions(pred_lines.detach().cpu().numpy(), (128, 128), (h, w))
    pred_lines = scale_positions(pred_lines.cpu().numpy(), (128, 128), (h, w))

    score = results[0]["scores"]
    line = pred_lines
    score_idx = np.argsort(-score)
    line = line[score_idx]
    score = score[score_idx]

    npz_save_path = npz_dir + f'{im_name}.npz'
    np.savez(npz_save_path, **{"lines": line, "line_scores": score})
    # break

In [ ]:

import os

In [ ]:
len(os.listdir('/home/kallelis/PrimitiveExtraction/PrimitiveExtraction/Detection/LETR/exp/exp/res101_stage2_focal/checkpoints/eida_final_npz'))

In [ ]:
remaining

In [ ]:

# # out_logits, out_line = outputs['pred_logits'], outputs['pred_lines']
# # prob = F.softmax(out_logits, -1)
# # scores, labels = prob[..., :-1].max(-1)
# # img_h, img_w = orig_size.unbind(0)
# # scale_fct = torch.unsqueeze(torch.stack([img_w, img_h, img_w, img_h], dim=0), dim=0).to('cuda')
# # lines = out_line * scale_fct[:, None, :]


# out_logits, out_line = outputs["pred_logits"], outputs["pred_lines"]

# # assert orig_size.shape[1] == 2

# prob = F.softmax(out_logits, -1)
# scores, labels = prob[..., :-1].max(-1)

# # convert to [x0, y0, x1, y1] format
# img_h, img_w = orig_size.unbind(0)
# scale_fct = torch.unsqueeze(torch.stack([img_w, img_h, img_w, img_h], dim=0), dim=0).to('cuda')
# lines = out_line * scale_fct[:, None, :]


# results = [
#     {"scores": s, "labels": l, "lines": b}
#     for s, l, b in zip(scores, labels, lines)
# ]

In [ ]:
results

In [ ]:
results[0]['scores'][-1]

In [ ]:
outputs["pred_logits"].shape

In [ ]:
# orig_size = torch.stack([orig_size] * 1, dim=0).to('cuda')
# results = postprocessors["line"](outputs, orig_size, "prediction", unbind_dim=1)
pred_logits = outputs["pred_logits"]
bz = pred_logits.shape[0]
assert bz == 1
query = pred_logits.shape[1]

rst = results[0]["lines"].detach()
pred_lines = rst.view(query, 2, 2)

# pred_lines = pred_lines.flip([-1])  # this is yxyx format

h, w = orig_size.tolist()
pred_lines[:, :, 0] = pred_lines[:, :, 0] * (128)
pred_lines[:, :, 0] = pred_lines[:, :, 0] / h
pred_lines[:, :, 1] = pred_lines[:, :, 1] * (128)
pred_lines[:, :, 1] = pred_lines[:, :, 1] / w

score = results[0]["scores"]
line = pred_lines.detach().cpu().numpy()

score_idx = np.argsort(-score)
line = line[score_idx]
score = score[score_idx]

In [ ]:
line

In [ ]:
score

In [ ]:
npz_dir = ckpt_path + f'/eida_final_npz/'
os.makedirs(npz_dir, exist_ok=True)
npz_save_path = npz_dir + f'{im_name}.npz'
np.savez(npz_save_path, **{"lines": line, "line_scores": score})

## Plot Inference Results

In [ ]:
keep = score >= 0.3
keep = keep.squeeze()
final_lines = lines[0][keep]
final_lines = final_lines.reshape(final_lines.shape[0], -1)
fig = plt.figure()
plt.imshow(raw_img)
for tp_id, line_to_plot in enumerate(final_lines):
    y1, x1, y2, x2 = line_to_plot.detach().cpu().numpy() # this is yxyx
    p1 = (x1, y1)
    p2 = (x2, y2)
    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], linewidth=1, color='red', zorder=1)
plt.axis('off')


#plt.savefig("../figures/demo_result.png", dpi=300, bbox_inches='tight', pad_inches = 0)
#plt.close(fig)
plt.show()
